# Import Commands

In [1]:
#%qtconsole
#%matplotlib inline

import matplotlib.pyplot as plt
from functools import partial
import numpy as np
import time 
import qcodes as qc
import os
import openpyxl

from qcodes import Station, load_or_create_experiment, initialise_database, Measurement, load_by_run_spec, load_by_guid, initialise_or_create_database_at 
from qcodes.utils.dataset.doNd import do1d, do2d, plot
from pyvisa import VisaIOError

from qcodes.instrument_drivers.stanford_research.SR830 import SR830
from qcodes.instrument_drivers.Keithley.Keithley_2450 import *
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\manfra-lab\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\manfra-lab\.qcodes\logs\231018-18116-qcodes.log


# Initialize Instruments

In [2]:
#Lockins(SR830)
lockIn = SR830('Gomorrah','GPIB0::8::INSTR' )
#SIM900 Controller(DC Voltage source)
keith = Keithley2450('Parrot', 'GPIB0::18::INSTR')
keith.reset()

Connected to: Stanford_Research_Systems SR830 (serial:s/n21560, firmware:ver1.01) in 0.20s
Connected to: KEITHLEY INSTRUMENTS 2450 (serial:04509597, firmware:1.7.5b) in 0.04s


# Global Parameters for Instruments and Experiment

In [3]:
dataBasePath = 'C:\\Users\\manfra-lab\\Desktop\\Users\\Matt\\Data\\QcodesTest\\test.db'

In [4]:
exp_name = "Differential_Conductance_100pA_Contact4_SweepDown"

In [5]:
meas_name = 'Differential_Conductance_100pA'

In [7]:
csvPath = "C:\\Users\\manfra-lab\\Desktop\\Users\\Matt\\Data\\QcodesTest\\test.csv"
sample = "8-3-22.1MM11A3"
#Sets the keithly to measure current and source voltage
keith.sense.function("current")
keith.source.function("voltage")
#Units are in volts
V_start = keith.source.voltage()
V_end = -0.5
#Ramp rate is volts/delay
rampRate = 0.1
#Time Delay(seconds)
delay = 1
#Changes direction of ramp if ramping down to a voltage instead of up
if V_end < V_start:
    rampRate = rampRate *(-1)
    
keith.output_enabled.set_to(True)
keith.terminals("front")

# Set Up Database for Data Collection

In [8]:
db_file_path = os.path.join(os.getcwd(), dataBasePath)
initialise_or_create_database_at(db_file_path)

# Set Up Live Plot

In [9]:
#Opens windows that will plot results as they are collected
#Make sure to turn on refresh rate once window opens or nothing will happen
jobs = bg.BackgroundJobManager()
jobs.new(inspectr.main, db_file_path)

<BackgroundJob #0: <function main at 0x000001E4061D1B80>>

# Setup Station and Measurement Parameters

In [10]:
#Handles equipment
station = qc.Station()
station.add_component(keith)
station.add_component(lockIn)

'Gomorrah'

In [11]:
#Manages independent and dependent variables
sweepKeith = load_or_create_experiment(
    experiment_name=exp_name,
    sample_name=sample
)
context_meas = Measurement(exp=sweepKeith, name=meas_name)
context_meas.write_period = 0.1

context_meas.register_parameter(keith.source.voltage)
context_meas.register_parameter(lockIn.X,setpoints=(keith.source.voltage,))

# Sweep Keithly and Measure Lockin With Context Manager

In [12]:
with context_meas.run() as datasaver:
    for v_set in np.arange(V_start, V_end+rampRate, rampRate):
        keith.source.voltage(v_set)
        get_x = lockIn.X.get()
        datasaver.add_result((keith.source.voltage, v_set),
                             (lockIn.X, get_x))        
        time.sleep(delay)
    dataset = datasaver.dataset

Starting experimental run with id: 15. 


# Converts Dataset Results to CSV

In [13]:
#Converts to pandas dataframe
df = dataset.to_pandas_dataframe()
#Converts dataframe to csv
df.to_csv(csvPath)